In [ ]:
# === Environment Setup ===
import os, sys, math, time, random, json, textwrap, warnings
import numpy as np, pandas as pd
from scipy.linalg import lu, lu_factor, lu_solve, solve_triangular, cholesky, qr, svd
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import spsolve
from scipy.datasets import face
import sympy as sp
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.patches import Polygon, Arrow
from IPython.display import display, Image, Markdown

# --- Configuration ---
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams.update({'figure.dpi': 130, 'font.size': 12, 'axes.titlesize': 'x-large',
    'axes.labelsize': 'large', 'xtick.labelsize': 'medium', 'ytick.labelsize': 'medium'})
np.set_printoptions(suppress=True, precision=4, linewidth=120)
sp.init_printing(use_unicode=True)

# --- Utility Functions ---
def note(msg, **kwargs):
    display(Markdown(f"<div class='alert alert-info'>📝 {textwrap.fill(msg, width=100)}</div>"))
def sec(title):
    print(f"\n{100*'='}\n| {title.upper()} |\n{100*'='}")

note("Environment initialized.")

# Part 2: Core Numerical Methods
## Chapter 2.1: Linear Algebra for Economists

### Introduction: The Language of Economic Systems

Linear algebra is the fundamental language used to express, analyze, and solve the interconnected systems that define modern economics. While microeconomics often begins with the study of a single agent or market, macroeconomics and econometrics are fundamentally concerned with systems of equations. From the market-clearing conditions of a general equilibrium model to the vast datasets of empirical work, linear algebra provides the tools to reason about entire systems through the geometry of vectors and matrices, rather than getting lost in a sea of individual equations.

This chapter provides a practical guide to the core concepts of linear algebra, focusing on geometric intuition and computational best practices. We will explore:
*   **Vector Spaces and Subspaces:** The abstract structures that govern vectors.
*   **The Four Fundamental Subspaces:** The cornerstone theorem linking a matrix's key spaces.
*   **Matrices as Transformations:** Understanding matrices as functions that transform vector spaces.
*   **Solving Systems:** The theory and robust methods for solving `Ax = b`, the workhorse of economic modeling.
*   **The Condition Number:** A critical measure of the numerical sensitivity of a system.
*   **Matrix Decompositions:** Unpacking matrix structure via LU, QR, Cholesky, and Eigendecomposition to solve problems efficiently and stably.
*   **Economic Applications:** Grounding these concepts in classic problems like the Leontief input-output model, OLS, and the stability of dynamic systems.

### 1. Vector Spaces, Subspaces, and Bases

A **vector space** is a collection of objects called vectors, which can be added together and multiplied by scalars. For a set to be a vector space, it must satisfy a specific set of axioms (closure under addition and scalar multiplication, existence of a zero vector, etc.). The familiar n-dimensional real space, $\mathbb{R}^n$, is the canonical example.

A **subspace** is a subset of a vector space that is itself a vector space. Crucially, it must contain the zero vector and be closed under vector addition and scalar multiplication. For example, a plane passing through the origin is a subspace of $\mathbb{R}^3$.

- **Linear Independence:** A set of vectors $\{v_1, ..., v_k\}$ is linearly independent if the only solution to $c_1v_1 + ... + c_kv_k = 0$ is $c_1 = ... = c_k = 0$. Intuitively, no vector in the set can be written as a linear combination of the others.
- **Span:** The span of a set of vectors is the set of all possible linear combinations of those vectors. The span of a set of vectors always forms a vector space (or a subspace).
- **Basis:** A basis for a vector space is a set of linearly independent vectors that span the space. A basis provides a unique coordinate system for the space.
- **Dimension:** The dimension of a vector space is the number of vectors in any basis for that space.

#### The Four Fundamental Subspaces

For any $m \times n$ matrix $A$, there are four fundamental subspaces that describe its properties. The **Fundamental Theorem of Linear Algebra** connects their dimensions and orthogonality.

1.  **Column Space, $C(A)$:** The span of the columns of $A$. It is a subspace of $\mathbb{R}^m$. Its dimension, $r$, is the **rank** of the matrix. The system $Ax=b$ is solvable if and only if $b$ is in $C(A)$.
2.  **Null Space, $N(A)$:** The set of all vectors $x$ such that $Ax = 0$. It is a subspace of $\mathbb{R}^n$. Its dimension is $n-r$.
3.  **Row Space, $C(A^T)$:** The span of the rows of $A$. It is a subspace of $\mathbb{R}^n$. Its dimension is also $r$.
4.  **Left Null Space, $N(A^T)$:** The set of all vectors $y$ such that $A^T y = 0$. It is a subspace of $\mathbb{R}^m$. Its dimension is $m-r$.

**Orthogonality:**
- The row space is orthogonal to the null space ($C(A^T) \perp N(A)$).
- The column space is orthogonal to the left null space ($C(A) \perp N(A^T)$).

In [ ]:
sec("Visualizing the Four Fundamental Subspaces")
# A 3x2 matrix maps from R^2 to R^3
A = np.array([[1, 2], [2, 4], [3, 1]])

# Calculate bases for the subspaces
# Column space and null space
U, s, Vt = svd(A)
rank = np.sum(s > 1e-10)
col_space_basis = U[:, :rank]
null_space_basis = Vt[rank:, :].T

# Row space and left null space
U_t, s_t, Vt_t = svd(A.T)
rank_t = np.sum(s_t > 1e-10)
row_space_basis = Vt_t[rank_t:, :].T # or V[:, :rank]
left_null_space_basis = U_t[:, rank_t:]

fig = plt.figure(figsize=(12, 6))
ax1 = fig.add_subplot(121, projection='3d') # Output space R^3
ax2 = fig.add_subplot(122) # Input space R^2

# Plotting Output Space (R^3)
xx, yy = np.meshgrid(np.linspace(-1, 1, 10), np.linspace(-1, 1, 10))
col_plane = col_space_basis[:, 0:1] * xx + col_space_basis[:, 1:2] * yy
ax1.plot_surface(col_plane[0,:,:], col_plane[1,:,:], col_plane[2,:,:], alpha=0.3, color='blue')
ax1.text(*col_space_basis[:,0]*0.7, 'C(A)', color='blue', fontsize=14)
q = ax1.quiver(0, 0, 0, *left_null_space_basis, color='red', length=1.5, normalize=True)
ax1.text(*left_null_space_basis.flatten()*1.2, 'N(A^T)', color='red', fontsize=14)
ax1.set_title('Output Space (R^3)')

# Plotting Input Space (R^2)
ax2.plot([-row_space_basis[0,0]*2, row_space_basis[0,0]*2], [-row_space_basis[1,0]*2, row_space_basis[1,0]*2], 'g-', lw=3, label='Row Space C(A^T)')
if null_space_basis.shape[1] > 0:
    ax2.plot([-null_space_basis[0,0]*2, null_space_basis[0,0]*2], [-null_space_basis[1,0]*2, null_space_basis[1,0]*2], 'm-', lw=3, label='Null Space N(A)')
ax2.set_xlim(-2, 2); ax2.set_ylim(-2, 2); ax2.set_aspect('equal'); ax2.grid(True)
ax2.set_title('Input Space (R^2)'); ax2.legend()

plt.tight_layout(); plt.show()

### 2. Matrices as Linear Transformations

A matrix $A \in \mathbb{R}^{m \times n}$ is a recipe for a **linear transformation**—a function that maps vectors from $\mathbb{R}^n$ to $\mathbb{R}^m$ while preserving the rules of vector addition and scalar multiplication. Geometrically, this means grid lines remain parallel and evenly spaced, and the origin stays fixed.

The **determinant** of a square matrix measures the factor by which the area (in 2D) or volume (in 3D) of a region changes after the transformation. A determinant of 0 means the transformation squashes space into a lower dimension (e.g., a plane into a line), and the matrix is **singular** (not invertible).

In [ ]:
sec("Visualizing a Matrix Transformation")

def plot_transformation(matrix, ax, title):
    i_hat, j_hat = np.array([1, 0]), np.array([0, 1])
    transformed_i, transformed_j = matrix @ i_hat, matrix @ j_hat
    ax.quiver(0, 0, *i_hat, color='gray', scale=1, scale_units='xy', angles='xy', label='$\hat{i}$')
    ax.quiver(0, 0, *j_hat, color='gray', scale=1, scale_units='xy', angles='xy', label='$\hat{j}$')
    ax.quiver(0, 0, *transformed_i, color='red', scale=1, scale_units='xy', angles='xy', label='$A\hat{i}$')
    ax.quiver(0, 0, *transformed_j, color='blue', scale=1, scale_units='xy', angles='xy', label='$A\hat{j}$')
    unit_square = np.array([[0,0], [1,0], [1,1], [0,1], [0,0]])
    ax.add_patch(Polygon((matrix @ unit_square.T).T, facecolor='lightblue', alpha=0.5))
    ax.set_xlim(-1, 4); ax.set_ylim(-1, 4); ax.set_aspect('equal', adjustable='box')
    ax.legend(); ax.grid(True); ax.set_title(f"{title}\nDeterminant = {np.linalg.det(matrix):.2f}")

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
A_shear = np.array([[1, 1.5], [0, 1]])
A_rotate_scale = np.array([[2, 1], [1, 2]])
plot_transformation(A_shear, ax1, 'Shear Transformation')
plot_transformation(A_rotate_scale, ax2, 'Rotation and Scale')
plt.tight_layout(); plt.show()

### 3. Solving Systems of Linear Equations: `Ax = b`
The problem of solving a system of linear equations is central to economics. Geometrically, solving `Ax = b` means asking: **"What vector `x`, after being transformed by the matrix `A`, lands on the vector `b`?"** 

A solution exists if and only if `b` is in the **column space** of `A`. If `A` is square and non-singular (its determinant is non-zero), a unique solution exists for any `b`.

**Best Practice:** Never compute the inverse `A⁻¹` directly to solve a system (i.e., avoid `x = inv(A) @ b`). Methods based on matrix decompositions, like those used in `np.linalg.solve`, are faster and far more numerically stable.

#### Application: The Leontief Input-Output Model
The Leontief model describes the interdependencies between sectors in an economy. If $A$ is the technology matrix where $A_{ij}$ is the value of input from sector $i$ needed to produce one unit of output in sector $j$, and $d$ is the final demand vector, then the gross output $x$ required to meet this demand is given by:
$$ x = Ax + d \implies (I - A)x = d $$
We need to solve this system for $x$. Since an economy's parameters might be analyzed under many different final demand scenarios ($d_1, d_2, ...$), this is a perfect case for using an efficient solver.

In [ ]:
sec("Solving the Leontief Input-Output Model")
# A: Technology Matrix (rows: input sector, cols: output sector)
# e.g., A[0,1] = 0.2 means Manuf needs 0.2 units from Ag for each unit of output
A = np.array([[0.1, 0.2, 0.4],  # Inputs from Agriculture
                [0.5, 0.1, 0.3],  # Inputs from Manufacturing
                [0.1, 0.6, 0.1]]) # Inputs from Services
I = np.eye(3)
L = I - A # The Leontief Matrix

# Decompose the Leontief matrix ONCE using lu_factor for efficiency.
lu_factors, piv = lu_factor(L)

note("The Leontief matrix (I - A) is LU-decomposed once for efficiency.")

def solve_leontief(d):
    """Solves the system for a given demand vector using the pre-computed LU factors."""
    return lu_solve((lu_factors, piv), d)

# Scenario 1: High demand for services
d1 = np.array([100, 200, 500])
x1 = solve_leontief(d1)
print(f"For demand d1 = {d1}, required gross output x1 = {x1}")

# Scenario 2: High demand for manufacturing
d2 = np.array([150, 600, 150])
x2 = solve_leontief(d2)
print(f"For demand d2 = {d2}, required gross output x2 = {x2}")

### 4. Stability and the Condition Number

The **condition number** of a matrix `A`, denoted $\kappa(A)$, measures how sensitive the solution of `Ax=b` is to small changes in `A` or `b`. A low condition number means the matrix is **well-conditioned**, and the solution is stable. A high condition number means the matrix is **ill-conditioned**, and small input errors can lead to large output errors.
$$ \kappa(A) = ||A|| \cdot ||A^{-1}|| $$
For a square matrix, $\kappa(A) = |\frac{\lambda_{max}}{\lambda_{min}}|$ where $\lambda$ are the eigenvalues. For OLS, the relevant condition number is that of the matrix $X^T X$. Severe multicollinearity in the design matrix $X$ leads to a very high condition number, making the OLS estimates unstable and unreliable.

In [ ]:
sec("Condition Number and Multicollinearity")
rng = np.random.default_rng(42)
n = 100
x1 = rng.standard_normal(n)

note("Case 1: Low correlation between x1 and x2")
x2_low_corr = 0.1 * x1 + rng.standard_normal(n) # Low correlation
X_low = np.c_[np.ones(n), x1, x2_low_corr]
cond_low = np.linalg.cond(X_low.T @ X_low)
print(f"Correlation: {np.corrcoef(x1, x2_low_corr)[0,1]:.2f}, Condition Number: {cond_low:.2f}")

note("Case 2: High correlation between x1 and x2")
x2_high_corr = 0.95 * x1 + 0.1 * rng.standard_normal(n) # High correlation
X_high = np.c_[np.ones(n), x1, x2_high_corr]
cond_high = np.linalg.cond(X_high.T @ X_high)
print(f"Correlation: {np.corrcoef(x1, x2_high_corr)[0,1]:.2f}, Condition Number: {cond_high:.2f}")
note("The extremely high condition number indicates that the OLS estimates will be very sensitive to small changes in the data.")

### 5. Special Matrices in Economics
Certain types of matrices appear frequently in economic theory and econometrics due to their special properties.

- **Symmetric Matrices ($A = A^T$):** Covariance matrices and Hessian matrices of second derivatives are always symmetric. Symmetric matrices have real eigenvalues and a full set of orthogonal eigenvectors.
- **Positive Definite Matrices:** A symmetric matrix $A$ is positive definite if $x^T A x > 0$ for all non-zero vectors $x$. This is a crucial property for covariance matrices (variance must be positive) and for ensuring a minimum in optimization problems.
- **Idempotent Matrices ($P^2 = P$):** Projection matrices, like the "hat" matrix in OLS ($P = X(X^TX)^{-1}X^T$), are idempotent. Applying a projection twice is the same as applying it once. Their eigenvalues are always 0 or 1.
- **Orthogonal Matrices ($Q^T Q = I$):** These matrices represent rotations and reflections. They preserve vector lengths and angles. They are fundamental to the QR decomposition and SVD.

In [ ]:
sec("Testing for Positive Definiteness")

note("A valid covariance matrix must be positive definite.")
valid_cov = np.array([[1, 0.5], [0.5, 1]])
try:
    # Cholesky decomposition only works for positive definite matrices
    cholesky(valid_cov)
    print("Matrix is positive definite (Cholesky succeeded).")
except np.linalg.LinAlgError:
    print("Matrix is not positive definite.")

note("An invalid covariance matrix is not positive definite.")
invalid_cov = np.array([[1, 1.1], [1.1, 1]]) # Correlation > 1
try:
    cholesky(invalid_cov)
    print("Matrix is positive definite.")
except np.linalg.LinAlgError as e:
    print(f"Matrix is not positive definite (Cholesky failed: {e})")

### 6. Eigenvalues and System Dynamics
An **eigenvector** `v` of a square matrix `A` is a special non-zero vector that does not change direction when transformed by `A`. It is only scaled by a factor, its corresponding **eigenvalue** `λ`: $Av = \lambda v$.

**Eigendecomposition** factors a matrix `A` into `A = PDP⁻¹`, where `P` is the matrix of eigenvectors and `D` is the diagonal matrix of eigenvalues. This is invaluable for understanding dynamic systems. If a system is described by the linear difference equation `x_{t+1} = Ax_t`, its long-term behavior is governed by the eigenvalues of `A`:
- If all $|\lambda_i| < 1$, the system is **stable** and converges to the origin.
- If any $|\lambda_i| > 1$, the system is **unstable** and diverges.
- If some $|\lambda_i| < 1$ and some $|\lambda_i| > 1$, the system has **saddle-path stability**.

In [ ]:
sec("Eigen-analysis of a 2D Dynamic System")

def plot_dynamics(A, ax, title):
    eigvals, _ = np.linalg.eig(A)
    ax.set_title(f"{title}\nEigenvalues: {eigvals[0]:.2f}, {eigvals[1]:.2f}", fontsize=10)
    ax.set_xlim(-5, 5); ax.set_ylim(-5, 5); ax.grid(True)
    
    for i in range(20):
        x0 = np.random.rand(2) * 4 - 2
        path = [x0]
        for t in range(15):
            x_next = A @ path[-1]
            path.append(x_next)
        path = np.array(path)
        ax.plot(path[:, 0], path[:, 1], '-o', markersize=2, alpha=0.7)

A_stable = np.array([[0.5, 0.2], [0.1, 0.7]])      # Both |lambda| < 1
A_unstable = np.array([[1.1, 0.2], [0.1, 1.3]])    # Both |lambda| > 1
A_saddle = np.array([[0.8, 0.3], [0.2, 1.2]])      # One |lambda| < 1, one |lambda| > 1

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))
plot_dynamics(A_stable, ax1, 'Stable System')
plot_dynamics(A_unstable, ax2, 'Unstable System')
plot_dynamics(A_saddle, ax3, 'Saddle-Path System')
plt.tight_layout()
plt.show()

### 7. The Singular Value Decomposition (SVD): The Master Tool

The SVD is arguably the most powerful and general matrix decomposition. It states that *any* matrix $A \in \mathbb{R}^{m \times n}$ can be factored as:
$$ A = U \Sigma V^T $$
Where:
- $U$ is an $m \times m$ orthogonal matrix whose columns (left singular vectors) are an orthonormal basis for the output space $\mathbb{R}^m$.
- $\Sigma$ is an $m \times n$ diagonal matrix containing the **singular values** $\sigma_i$ (which are non-negative and sorted).
- $V^T$ is an $n \times n$ orthogonal matrix whose rows (right singular vectors) are an orthonormal basis for the input space $\mathbb{R}^n$.

Geometrically, the SVD states that any linear transformation can be broken down into three fundamental operations: a rotation (by $V^T$), a scaling along orthogonal axes (by $\Sigma$), and another rotation (by $U$). Its power comes from its generality—it applies to any matrix, square or not—and its ability to reveal the underlying structure of the data.

#### Application: Principal Component Analysis (PCA)
PCA is a cornerstone of dimensionality reduction. Given a dataset $X$, PCA finds the directions of maximum variance (the principal components). These components are precisely the right singular vectors ($V$) of the centered data matrix. The SVD provides a direct and numerically stable way to compute them.

In [ ]:
sec("Dimensionality Reduction with SVD (PCA)")
rng = np.random.default_rng(123)
# Create correlated data
X_orig = rng.multivariate_normal([0, 0], [[1, 0.8], [0.8, 1]], size=200)

# 1. Center the data
X_centered = X_orig - X_orig.mean(axis=0)

# 2. Compute SVD
U, s, Vt = np.linalg.svd(X_centered)
V = Vt.T # V contains the principal components as columns

# 3. Project data onto the first principal component
X_projected = X_centered @ V[:, 0]

note("The first column of V is the direction of maximum variance.")
plt.figure(figsize=(8, 6))
plt.scatter(X_centered[:, 0], X_centered[:, 1], alpha=0.6, label='Original Centered Data')
# Plot the principal component vectors, scaled by singular values
plt.quiver(0, 0, V[0, 0]*s[0], V[1, 0]*s[0], color='r', scale=1, scale_units='xy', angles='xy', label='PC1')
plt.quiver(0, 0, V[0, 1]*s[1], V[1, 1]*s[1], color='g', scale=1, scale_units='xy', angles='xy', label='PC2')
plt.xlabel('X1'); plt.ylabel('X2'); plt.legend(); plt.axis('equal'); plt.grid(True)
plt.title('Principal Components via SVD')
plt.show()

### 8. Sparse Matrices in Economics

Many economic models, especially those involving networks, panel data with fixed effects, or large-scale structural models, produce matrices that are very large but **sparse**—meaning most of their elements are zero. Storing these matrices as dense NumPy arrays is incredibly inefficient in terms of both memory and computation.

The `scipy.sparse` module provides data structures and algorithms optimized for sparse matrices. Instead of storing all $N \times M$ elements, they only store the non-zero values and their locations.

In [ ]:
sec("Memory and Speed of Sparse Matrices")
N = 10000
# Create a sparse matrix (e.g., a simple tridiagonal matrix)
diag = np.ones(N) * -2
off_diag = np.ones(N - 1)
diagonals = [diag, off_diag, off_diag]
offsets = [0, -1, 1]
A_sparse = csr_matrix((np.concatenate(diagonals), (np.concatenate([np.arange(N), np.arange(N-1), np.arange(1,N)]), np.concatenate([np.arange(N), np.arange(1,N), np.arange(N-1)]))), shape=(N, N))
A_dense = A_sparse.toarray()

note(f"Memory of dense {N}x{N} matrix: {A_dense.nbytes / 1e6:.2f} MB")
note(f"Memory of sparse matrix: {A_sparse.data.nbytes / 1e6:.2f} MB (plus overhead)")

# Solve Ax=b
b = np.random.rand(N)
note("Solving Ax=b using a sparse solver is much faster.")
%timeit spsolve(A_sparse, b)
note("Solving with a dense solver is slower.")
%timeit np.linalg.solve(A_dense, b)

### 9. Chapter Summary

- **Foundation:** Linear algebra is built on the concepts of vector spaces, subspaces, basis, and dimension. The **Four Fundamental Subspaces** provide a complete picture of a matrix's action.
- **Geometric Intuition:** Matrices are linear transformations that rotate, scale, and shear space. The determinant measures the change in volume, and eigenvalues/eigenvectors describe the directions that are unchanged by the transformation.
- **Solving Systems:** The workhorse problem `Ax=b` should be solved with decomposition-based methods like `np.linalg.solve` (which uses LU) or `scipy.linalg.lu_solve` for repeated solves. **Never use `inv(A)`**.
- **Numerical Stability:** The **condition number** is a vital diagnostic for the sensitivity of `Ax=b`. For OLS, this reveals multicollinearity. The **QR decomposition** provides the most numerically stable method for solving least-squares problems.
- **Decompositions are Key:** Understanding LU, QR, Cholesky, Eigendecomposition, and SVD is crucial. Each reveals a different aspect of the matrix and is suited to different problems (solving systems, stability analysis, dimensionality reduction).
- **Sparsity:** Real-world economic models often generate huge, sparse matrices. Using `scipy.sparse` is essential for memory and computational efficiency.

### 10. Exercises

1.  **Proof of Eigenvalues:** Prove that the eigenvalues of a real, symmetric, idempotent matrix must be either 0 or 1.
    - *Hint: Start with the definition $Px = \lambda x$. Multiply both sides by $P$. Use the idempotent property $P^2=P$.*

2.  **QR Decomposition for OLS:** A more stable way to solve the OLS problem is to use QR decomposition on the design matrix $X$. If $X=QR$, the normal equations $(X^T X)\beta = X^T y$ become $(R^T Q^T Q R)\beta = R^T Q^T y$, which simplifies to $R\beta = Q^T y$ since $Q^T Q = I$. This is an upper-triangular system that can be solved efficiently with back substitution.
    - **Task:** Using the data from the condition number example, compute the QR decomposition of `X_high`. Then solve the system $R\beta = Q^T y$ for $\beta$ using `scipy.linalg.solve_triangular`. Compare your result to the standard `np.linalg.lstsq` solution.

3.  **SVD for Image Compression:** A powerful application of low-rank approximation via SVD is image compression. An image can be represented as a matrix of pixel values. By taking the SVD of the image matrix and reconstructing it using only the top `k` singular values and vectors, we can achieve significant compression.
    - **Task:** Load an image (e.g., from `scipy.datasets.face(gray=True)`). Compute its SVD. Write a function that reconstructs the image using only the top `k` singular values. Use Matplotlib to display the original image and several reconstructions with different values of `k` (e.g., 5, 20, 50) to see the trade-off between compression and image quality.

4.  **Sparse Fixed Effects:** In panel data econometrics, fixed effects are often implemented by creating a large number of dummy variables, one for each individual or group. This results in a very large, very sparse design matrix. 
    - **Task:** Create a sparse matrix representing a design matrix with 100,000 observations and 10,000 individual fixed effects (so it's a 100,000 x 10,000 matrix). Each row should have exactly one '1' in one of the 10,000 columns. Compare the memory usage of this matrix when stored as a `scipy.sparse.csr_matrix` versus a dense NumPy array.